In [1]:
from sklearn import svm
import collections
import numpy as np
import scipy as sc
 

In [2]:
#amino acid sinle letter codes
aac = {}

rest = 'Z';

codes = np.eye(21)
aac['G'] = codes[0]
aac['A'] = codes[1]
aac['L'] = codes[2]
aac['M'] = codes[3]
aac['F'] = codes[4]
aac['W'] = codes[5]
aac['K'] = codes[6]
aac['Q'] = codes[7]
aac['E'] = codes[8]
aac['S'] = codes[9]
aac['P'] = codes[10]
aac['V'] = codes[11]
aac['C'] = codes[12]
aac['I'] = codes[13]
aac['Y'] = codes[14]
aac['H'] = codes[15]
aac['R'] = codes[16]
aac['N'] = codes[17]
aac['D'] = codes[18]
aac['T'] = codes[19]
aac[rest] = codes[20]

alphabeth = ['H', 'E', 'C']
alph = {}
alph['H'] = 0
alph['E'] = 1
alph['C'] = 2

In [3]:
# prepare training data for 3-class SVM
def convert_input(sec, prim, sw):
    length = len(prim) - 1
    ins = []
    outs = []
    for j in range(length - sw):
        inputs = []
        for k in range(j, j+sw):
            if aac.has_key(prim[k]):
                inputs.append(aac[prim[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        outs.append(alph[sec[j+sw/2]])
    return (ins, outs)

In [10]:
# sliding window
sw = 9
# number of samples for file
x = 50

inputs_train = []
outputs_train = []

#read dataset file
f = open('dataset.txt', 'r')
for i in range(x):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs = convert_input(sec, prim, sw)
    for q in ins:
        inputs_train.append(q)
    for q in outs:
        outputs_train.append(q)
    
f.close()

print len(inputs_train)
print len(outputs_train)

clf = svm.SVC(C=1.5, gamma=0.1)
clf.fit(inputs_train, outputs_train)

9101
9101


SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [5]:
def display_result(outputs, alphabet):
    result = ''
    for x in outputs:
        result += alphabet[x]
    return result

def compare(a, b, s):
    l = len(b) - 1
    x = 0.0
    y = 0.0
    for i in range(l - s):
        if a[i] == b[i + s/2]:
            x += 1
        else:
            y += 1
    return x/(x+y)

In [18]:
f = open('dataset.txt', 'r')

sum = 0
z = 100

for i in range(x+z):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    
    if i > x:
        ins, outs = convert_input(sec, prim, sw)
        pred = display_result(clf.predict(np.array(ins, np.float32)), ['H', 'E', 'C'])
        sum += compare(pred, sec, sw)

print "total"
print sum/z

total
0.598975345141


In [49]:
def prepare_input_for(sec, prim, sw, struct):
    length = len(prim) - 1
    prefix = ""
    sufix = ""
    for q in range(sw/2):
        prefix += rest
        sufix += rest
    primx = prefix + prim[0:-1] + sufix
    ins = []
    outs = []
    for j in range(length):
        inputs = []
        for k in range(j - sw/2, j + sw/2 + 1):
            if aac.has_key(primx[k]):
                inputs.append(aac[primx[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        if sec[j] == struct:
            outs.append(1)
        else:
            outs.append(0)
    return (ins, outs)

In [46]:
def prepare_input_for2(sec, prim, sw, sx, sy):
    length = len(prim) - 1
    
    sec1 = ""
    prefix = ""
    sufix = ""
    for q in range(sw/2):
        prefix += rest
        sufix += rest
    primx = prefix + prim[0:-1] + sufix
    
    ins = []
    outs = []
    for j in range(length):
        if sec[j] != sx and sec[j] != sy:
            continue
        inputs = []
        for k in range(j - sw/2, j + sw/2 + 1):
            if aac.has_key(primx[k]):
                inputs.append(aac[primx[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        if sec[j] == sx:
            outs.append(1)
        else:
            outs.append(0)
        sec1 += sec[j]
    return (ins, outs, sec1)

In [47]:
def calcQ(pred, sec, struct):
    correct = 1
    observed = 1
    
    for i in range(len(pred)):
        if sec[i] == struct:
            observed += 1
            if sec[i] == pred[i]:
                correct += 1
    
    return 100 * correct / observed

def calcQpred(pred, sec, struct):
    correct = 1
    observed = 1
    
    for i in range(len(pred)):
        if pred[i] == struct:
            observed += 1
            if pred[i] == sec[i]:
                correct += 1
    
    return 100 * correct / observed

In [50]:
inputs_train = []
outputs_train = []

dssp = 'C'
z = 50

f = open('dataset.txt', 'r')
for i in range(z):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs = prepare_input_for(sec, prim, sw, dssp)
    for q in ins:
        inputs_train.append(q)
    for q in outs:
        outputs_train.append(q)
    
f.close()

clf_H = svm.SVC(C=1.5, gamma=0.1)
clf_H.fit(inputs_train, outputs_train)

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
f = open('dataset.txt', 'r')

for i in range(z+10):
    f.readline()
    prim = f.readline()
    sec = f.readline()

    ins, outs = prepare_input_for(sec, prim, sw, dssp)

    pred = display_result(clf_H.predict(np.array(ins, np.float32)), {0:'X', 1:dssp})
    if i > z:
        print str(calcQ(pred, sec, dssp)) + " : " + str(calcQpred(pred, sec, dssp))

51 : 77
32 : 57
31 : 51
36 : 62
25 : 53
53 : 56
30 : 60
32 : 45
29 : 77


In [42]:
inputs_train = []
outputs_train = []

sx = 'H'
sy = 'E'
z = 50

f = open('dataset.txt', 'r')
for i in range(z):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs, sec1 = prepare_input_for2(sec, prim, sw, sx, sy)
    for q in ins:
        inputs_train.append(q)
    for q in outs:
        outputs_train.append(q)
    
f.close()

clf_HE = svm.SVC(C=1.5, gamma=0.1)
clf_HE.fit(inputs_train, outputs_train)

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
f = open('dataset.txt', 'r')

for i in range(z+10):
    f.readline()
    prim = f.readline()
    sec = f.readline()

    ins, outs, sec = prepare_input_for2(sec, prim, sw, sx, sy)

    pred = display_result(clf_HE.predict(np.array(ins, np.float32)), {0:sy, 1:sx})
    if i > z:
        print sec
        print pred
        print "\n"

EEEEEEEEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEHHHHHHHHHHHEEEEHHHHHHHHHHHHHHEEHHHHHHHHHHHHHHHHEEEEEHHHHHHHHHHEEEEEHHHHHHHEEEHHHHHHHHHH
HHHHHHEHHHEEHHHEHHHHHHHHHHHHHHEHEEHHHHHHHHHEEHHHHHHHHHEHHHHHHHHHHHHEEEEHHHEHHHHHHEHHHHHHHHHEHEEHHHHHHHHHEEEEHHHEEEHEHHEEHHHHHHHHHHHHHHHHHHHHHHH


EEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEHHHHHHHHHHHHHHHHHHHHHHHHHHHEHHHHHHHHHHHHHHHHHHHHH
EEHHEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEEHHHHHHHHEEEHEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEHHHHHHHHHHHHHHHHHHHHEEEEEHHHHHHHHHEHHHHHHHHH


EEEEEHHHHHHHHHHHHEEEEEEEEEEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
EEEEHHHHHHHHHHHHHEEHEHEEEHHHHHHHHHHHHHHHHHHHEEHHHHEHHHHHHHHHHHHHHEEEEHHHHHHHHHHHHHHHHHHHEHHHHHHHHHHHHHHHHHHHEHHHHHHHHHHHHHHEHHHHHHHHHHHHH


EEEEEHHHHHHHHHHHHHHHEHHHHHHHHEEEEEEEEEEEEEEEEEHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEEEEEEEEHHHHHHHHHHHHHHHHEEEEEEHHHHHHHHHEEEEEEEHHHEEEEEEEEE
EEEHEHH